In [43]:
#import necessary libraries
import pandas as pd
import glob
import sagemaker
from sagemaker.predictor import Predictor
import json
import boto3
from sklearn.metrics import classification_report

The data for this model is not ready to be used in a model before we can make it available for model training.
1) Download data from http://www.consultaesic.cgu.gov.br/busca/_layouts/15/DownloadPedidos/DownloadDados.aspx
There is no bulk download and you will need to repeat the process for each year. In this case from 2015 to 2022. For each year we are only interested in the the files that have 'Pedidos' in its name.
The csv files downloaded will have the date when they were downloaded as their prefixes in the formar yyyymmdd

2) Upload the file in the data folder

In [3]:
date_of_download = '20220219'

In [4]:
# compile all files into one
pedidos_path = glob.glob(f"../data/{date_of_download}_Pedidos_csv_*")

files_list = []

for filename in pedidos_path:
    
    pedidos_file = pd.read_csv(filename,
                               sep=";",
                               quotechar="'",
                        header = None,
                        names=['IdPedido',
                              'ProtocoloPedido',
                              'Esfera',
                              'OrgaoDestinatario',
                              'Situacao',
                              'DataRegistro',
                              'ResumoSolicitacao',
                              'DetalhamentoSolicitacao',
                              'PrazoAtendimento',
                              'FoiProrrogado',
                              'FoiReencaminhado',
                              'FormaResposta',
                              'OrigemSolicitacao',
                              'IdSolicitante',
                              'AssuntoPedido',
                              'SubAssuntoPedido',
                              'Tag',
                              'DataResposta',
                              'Resposta',
                              'Decisao',
                              'EspecificacaoDecisao'],
                               keep_default_na=False,
                        encoding='UTF-16 LE')
    
    files_list.append(pedidos_file)

pedidos = pd.concat(files_list, axis=0, ignore_index=True)

In [5]:
pedidos.head()

,IdPedido,ProtocoloPedido,Esfera,OrgaoDestinatario,Situacao,DataRegistro,ResumoSolicitacao,DetalhamentoSolicitacao,PrazoAtendimento,FoiProrrogado,...,FormaResposta,OrigemSolicitacao,IdSolicitante,AssuntoPedido,SubAssuntoPedido,Tag,DataResposta,Resposta,Decisao,EspecificacaoDecisao
0,1493014,1390000001201816,Federal,ANATEL – Agência Nacional de Telecomunicações,Concluída,01/01/2018,Medicao de irradiacao de antenas de celular e ...,Senhores gostaria que fosse realizada medicao ...,22/01/2018,Não,...,Pelo sistema (com avisos por email),Internet,2847071,Outros em Pesquisa e Desenvolvimento,,,09/01/2018,"Prezada Senhora, Reportamo-nos à solicitação r...",Não se trata de solicitação de informação,
1,1493015,23480000001201831,Federal,IBGE – Fundação Instituto Brasileiro de Geogra...,Concluída,01/01/2018,"tenho que fazer um artigo , estou escrevendo s...",Oi boa tarde! Estou fazendo um artigo e não es...,22/01/2018,Não,...,Pelo sistema (com avisos por email),Internet,2847072,Direitos Humanos,,,19/01/2018,"Prezada Senhora Ana Paula, Como a sua solicita...",Acesso Concedido,Orientação sobre como encontrar a informação s...
2,1493016,12649000001201855,Federal,SUSEP – Superintendência de Seguros Privados,Concluída,01/01/2018,Informação do processo SUSEP,Regulamento do seguro com nº de processo SUSEP...,22/01/2018,Não,...,Pelo sistema (com avisos por email),Internet,2416000,Outros em Economia e Finanças,,,17/01/2018,"Prezado senhor, seguem, em anexo, as Condições...",Acesso Concedido,Resposta solicitada inserida no Fala.Br
3,1493017,37400000001201899,Federal,INSS – Instituto Nacional do Seguro Social,Concluída,01/01/2018,beneficio aposentadoria do mês de dezembro vei...,Gostaria de saber porque o meu beneficio de ap...,22/01/2018,Não,...,Pelo sistema (com avisos por email),Internet,2144003,Outros em Previdência,,,02/01/2018,Prezado(a) Senhor(a). A sua solicitação trata-...,Acesso Negado,Dados pessoais
4,1493018,99923000001201847,Federal,ECT – Empresa Brasileira de Correios e Telégrafos,Concluída,01/01/2018,PASEP,Gostaria de saber porque a RAIS referente ao m...,22/01/2018,Não,...,Pelo sistema (com avisos por email),Internet,2144003,Acesso à informação,,,22/01/2018,"Prezado Senhor Nestor Meirelles Filho, Em at...",Acesso Concedido,Resposta solicitada inserida no Fala.Br


In [6]:
#transfoming into appropriate date time
pedidos['DataRegistro_formatted'] =  pd.to_datetime(pedidos.DataRegistro,errors='coerce', format="%d/%m/%Y")
pedidos['PrazoAtendimento_formatted'] =  pd.to_datetime(pedidos.PrazoAtendimento, errors='coerce',format="%d/%m/%Y")
pedidos['DataResposta_formatted'] =  pd.to_datetime(pedidos.DataResposta,errors='coerce', format="%d/%m/%Y")

In [7]:
#removing incorrectly formatted rows
pedidos.dropna(inplace=True)

In [8]:
pedidos.shape

(628180, 24)

In [9]:
#Erasing some rows with bad formatting
pedidos = pedidos[ ~ (pedidos.Decisao == 'Gostaria de solicitar os Custos Operacionais para os anos de 2012, 2013 e 2014(se possível separados por Pessoal,Serviço de Terceiros e Materiais) com os seus respectivos comprimentos de rede(km), Potência(MVA), Quantidade de Transformadores e Quantidade de Módulos para as transmissoras que seguem abaixo: • CEEE • CEMIG • CHESF • COPEL • CTEEP • ELETRONORTE • ELETROSUL • FURNAS As informações referentes aos anos de 2002 à 2008 estão contidas na NT396/2009 SRE ANEEL(com custos anualizados referentes à junho/2009) e as informações referente aos anos de 2007 à 2011 na NT383/2012(com custos anualizados referentes à outubro/2012. Por fim, gostaria de solicitar as seguintes informações complementares: - Extensão das Linhas de Transmissão por nível de tensão (km) - Área de atuação das empresas em 2014 (km2) - Histórico da razão entre Parcela Variável e Receita por transmissora conforme o banco de dados disponível.  (Por exemplo: 2007-2008,2008-2009,2009-2010,2010-2011,                2011-2012,2012-2013,2013-2014) ')]

The field "Decisao" contains the agency status after a responde was given

In [10]:
set(pedidos.Decisao)

{'Acesso Concedido',
 'Acesso Negado',
 'Acesso Parcialmente Concedido',
 'Informação Inexistente',
 'Não se trata de solicitação de informação',
 'Pergunta Duplicada/Repetida',
 'Órgão não tem competência para responder sobre o assunto'}

In [11]:
pedidos.groupby('Decisao').count()

,IdPedido,ProtocoloPedido,Esfera,OrgaoDestinatario,Situacao,DataRegistro,ResumoSolicitacao,DetalhamentoSolicitacao,PrazoAtendimento,FoiProrrogado,...,IdSolicitante,AssuntoPedido,SubAssuntoPedido,Tag,DataResposta,Resposta,EspecificacaoDecisao,DataRegistro_formatted,PrazoAtendimento_formatted,DataResposta_formatted
Decisao,,,,,,,,,,,,,,,,,,,,,
Acesso Concedido,443478,443478,443478,443478,443478,443478,443478,443478,443478,443478,...,443478,443478,443478,443478,443478,443478,443478,443478,443478,443478
Acesso Negado,41146,41146,41146,41146,41146,41146,41146,41146,41146,41146,...,41146,41146,41146,41146,41146,41146,41146,41146,41146,41146
Acesso Parcialmente Concedido,30862,30862,30862,30862,30862,30862,30862,30862,30862,30862,...,30862,30862,30862,30862,30862,30862,30862,30862,30862,30862
Informação Inexistente,22287,22287,22287,22287,22287,22287,22287,22287,22287,22287,...,22287,22287,22287,22287,22287,22287,22287,22287,22287,22287
Não se trata de solicitação de informação,56425,56425,56425,56425,56425,56425,56425,56425,56425,56425,...,56425,56425,56425,56425,56425,56425,56425,56425,56425,56425
Pergunta Duplicada/Repetida,13711,13711,13711,13711,13711,13711,13711,13711,13711,13711,...,13711,13711,13711,13711,13711,13711,13711,13711,13711,13711
Órgão não tem competência para responder sobre o assunto,20270,20270,20270,20270,20270,20270,20270,20270,20270,20270,...,20270,20270,20270,20270,20270,20270,20270,20270,20270,20270


'Acesso Concedido' -> Granted

 'Acesso Negado' -> Denied
 
 'Acesso Parcialmente Concedido' -> Partially granted
 
 'Informação Inexistente' -> Information non existent
 
 'Não se trata de solicitação de informação' -> It it not a request for information
 
 'Pergunta Duplicada/Repetida' -> duplicated request
 
 'Órgão não tem competência para responder sobre o assunto -> The agency does not have the capability to answer request

This will be the target variable, but we won't use all the labels as they can be a bit misleading. In the end we want a model that can predict the odds of being granted or denied and its probability. So we will remove 'partially granted', 'information non existent', 'duplicated request' and 'The agency does not have the capability to answer request'. We will also combine 'Denied' with 'It is not a request for information.

In [12]:
pedidos = pedidos[~pedidos.Decisao.isin(['Acesso Parcialmente Concedido',
                              'Informação Inexistente',
                              'Pergunta Duplicada/Repetida',
                              'Órgão não tem competência para responder sobre o assunto'])
       ]

In [13]:
pedidos.shape

(541049, 24)

In [14]:
pedidos.loc[(pedidos.Decisao == 'Acesso Negado') | (pedidos.Decisao == 'Não se trata de solicitação de informação'),'target'] = 0
pedidos.loc[(pedidos.Decisao == 'Acesso Concedido'),'target'] = 1


In [15]:
pedidos['target'] = pedidos.target.astype('int')

In [16]:
pedidos.groupby('target').count()

,IdPedido,ProtocoloPedido,Esfera,OrgaoDestinatario,Situacao,DataRegistro,ResumoSolicitacao,DetalhamentoSolicitacao,PrazoAtendimento,FoiProrrogado,...,AssuntoPedido,SubAssuntoPedido,Tag,DataResposta,Resposta,Decisao,EspecificacaoDecisao,DataRegistro_formatted,PrazoAtendimento_formatted,DataResposta_formatted
target,,,,,,,,,,,,,,,,,,,,,
0,97571,97571,97571,97571,97571,97571,97571,97571,97571,97571,...,97571,97571,97571,97571,97571,97571,97571,97571,97571,97571
1,443478,443478,443478,443478,443478,443478,443478,443478,443478,443478,...,443478,443478,443478,443478,443478,443478,443478,443478,443478,443478


In [17]:
pedidos['request_length'] = pedidos.DetalhamentoSolicitacao.apply(lambda x : len(x))

In [18]:
pedidos = pedidos[pedidos.request_length > 10]

In [19]:
pedidos.shape

(539776, 26)

In [20]:
data = pedidos.groupby('target').apply(lambda x: x.sample(n=5000,random_state=42)).drop('target',axis=1).reset_index()

In [21]:
# The data needs to be formated in a specific way to be used by the model
data[['target','DetalhamentoSolicitacao']].to_csv('../data/data.csv',index=False,header=False)

In [22]:
!aws s3 cp ../data/data.csv s3://public-information-access/train/

upload: ../data/data.csv to s3://public-information-access/train/data.csv


After uploading the data, junmpstart was used to train the model 'BERT Base Multilingual Cased' and deploye it. The next step is to test it

In [23]:
text1 = 'gostaria de saber o numero do meu telefone'
text2 = 'qual a lista de funcionarios do detran'

In [24]:
newline, bold, unbold = '\n', '\033[1m', '\033[0m'
def query_endpoint(encoded_text):
    endpoint_name = 'jumpstart-ftc-eval-v1'
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/x-text', Body=encoded_text, Accept='application/json;verbose')
    return response

def parse_response(query_response):
    model_predictions = json.loads(query_response['Body'].read())
    probabilities, labels, predicted_label = model_predictions['probabilities'], model_predictions['labels'], model_predictions['predicted_label']
    return probabilities, labels, predicted_label

for text in [text1, text2]:
    query_response = query_endpoint(text.encode('utf-8'))
    probabilities, labels, predicted_label = parse_response(query_response)
    print (f"Inference:{newline}"
            f"Input text: '{text}'{newline}"
            f"Model prediction: {probabilities}{newline}"
            f"Labels: {labels}{newline}"
            f"Predicted Label: {bold}{predicted_label}{unbold}{newline}")

Inference:
Input text: 'gostaria de saber o numero do meu telefone'
Model prediction: [0.9178364801917611, 0.08216351980823881]
Labels: [0, 1]
Predicted Label: 0

Inference:
Input text: 'qual a lista de funcionarios do detran'
Model prediction: [0.35187930510197385, 0.6481206948980262]
Labels: [0, 1]
Predicted Label: 1



In [34]:
data_test = pedidos[~pedidos.IdPedido.isin(data.IdPedido.to_list())].sample(n=1000,random_state=42)

In [42]:
data_test['target_predicted'] = data_test.DetalhamentoSolicitacao.apply(lambda x: parse_response(query_endpoint(x.encode('utf-8')))[2])

In [44]:
print(classification_report(data_test.target, data_test.target_predicted, target_names=['denied','granted']))

              precision    recall  f1-score   support

      denied       0.38      0.60      0.47       170
     granted       0.91      0.80      0.85       830

    accuracy                           0.77      1000
   macro avg       0.64      0.70      0.66      1000
weighted avg       0.82      0.77      0.78      1000



We can see that the model does not perform so well for the denied class even though the training set is balanced. For the purposes of this demonstration we won't pursue further finetuning, as to avoid costs on training the model.

## Lambda

With the model deployed it is possible to create a lambda function to work as an API. The file 'lambda.py' has the code